## Weight & Biases
Zur Nachverfolgung und Analyse der Model-Performance

### Dataset
- **Train:** Modell wird trainiert & validiert. Modell & Pipeline serialisiert
- **Test:** Serialisierte Pipeline & Modell wird auf Test-Set angewendet

In [5]:
train_run = False

### Initialisieren

In [6]:
from typing import Final

import pandas as pd
import wandb
from joblib import load
from numpy import mean
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

import src.package.importer as im
import src.package.importer_usages as imp_usg
import src.package.ml_helper as ml_helper
import src.package.shared as sh
import src.package.consts as c
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error, mean_absolute_percentage_error

pd.set_option('display.max_rows', 500)  # to show all value_counts

MODEL_GRADIENT_BOOSTING: Final = 'GradientBoosting'
MODEL_LINEAR_REGRESSION: Final = 'LinearRegression'
current_model = MODEL_GRADIENT_BOOSTING

TAG_TRAIN_SET: Final = 'train-set'
TAG_TEST_SET: Final = 'test-set'

def load_dataset():
    set_file = 'train_set.csv' if train_run else 'test_set.csv'
    df = im.get_extended_dataset(f'../package/datasets/{set_file}')
    df, usage_types = imp_usg.extract_usage_details(df, highest_only=True, combine_garages=False)

    features = [
            c.FIELD_AREA_MAIN_USAGE,
            c.FIELD_AREA_TOTAL_FLOOR_416,
            c.FIELD_USAGE_CLUSTER,
            # c.FIELD_NOM_USAGE_MAIN,
            c.FIELD_NUM_FLOORS_UNDERGROUND,
            c.FIELD_NUM_FLOORS_OVERGROUND,
            c.GARAGE_TYPE_UG,
            c.GARAGE_TYPE_OG,
            c.FIELD_TOTAL_EXPENSES,
            c.PRIMARY_USAGE_PERCENTAGE,
            c.SECONDARY_USAGE_PERCENTAGE,
            c.TERTIARY_USAGE_PERCENTAGE,
            c.QUATERNARY_USAGE_PERCENTAGE,
            c.FIELD_VOLUME_TOTAL_416,
            c.FIELD_VOLUME_TOTAL_116
        ]
    # features.extend(usage_types)
    # features.append(c.GARAGE_COMBINED)

    if train_run:
        df = ml_helper.remove_outliers(df, factor=2.3)
        return ml_helper.hnf_dataset_full(df, features=features)
    else:
        # load transform pipeline for test_set
        pipeline = load(sh.export_directory_path(f'fitted_pipeline.joblib'))
        return ml_helper.hnf_dataset_full(df, fitted_pipeline=pipeline)

def init_wandb(X, tag_set, model, related_run=None):
    wandb.login()

    config = {
        'model': model,
        'features': ', '.join(X.columns.values),
        'dataset-count': len(X.index),
        'description': 'Impute volume 416 (clustered), Removed outliers (2.3), Drop all, OneHotEncoding, Default Hyperparameter'
    }

    if related_run is not None:
        config['related_run'] = related_run

    print(config)
    wandb.init(project='Metriken Bauwesen', entity='devcore', config=config, tags=[tag_set])

def wandb_plot_regressor(regr, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    regr.fit(X_train, y_train)

    wandb.sklearn.plot_regressor(regr, X_train, X_test, y_train, y_test)

### Train-Run

In [7]:
if train_run:
    X, y = load_dataset()
    #init_wandb(X, TAG_TRAIN_SET, current_model)

    regr = GradientBoostingRegressor(random_state=0)
    scores_map = ml_helper.cross_validation(regr, X, y)

    # log result of cross validation to wandb
    for key in scores_map.keys():
        if key in {'fit_time', 'score_time'}:
            continue
        scores = scores_map[key]
        # wandb.log({f'{key}_mean': mean(scores)})
        print({f'{key}_mean': mean(scores)})

    #wandb_plot_regressor(regr, X, y)

    # serialize model for test set
    regr.fit(X, y)
    sh.serialize_object(regr, 'fitted_model')

### Test-Run

In [8]:
if not train_run:
    X, y = load_dataset()
    # init_wandb(X, TAG_TEST_SET, current_model, related_run='peach-universe-66')

    regr = load(sh.export_directory_path(f'fitted_model.joblib'))

    predictions = regr.predict(X)
    r2_score = regr.score(X, y)
    print("R^2 value: ", r2_score)
    # wandb.log({'test_r2_mean': r2_score})

    neg_mean_squared_error = -mean_squared_error(y, predictions, squared=False)
    print("neg_mean_squared_error", neg_mean_squared_error)
    # wandb.log({'test_neg_root_mean_squared_error_mean': neg_mean_squared_error})

    neg_mean_absolute_error = -mean_absolute_error(y, predictions)
    print("neg_mean_absolute_error", neg_mean_absolute_error)
    # wandb.log({'test_neg_mean_absolute_error_mean': neg_mean_absolute_error})

    neg_mape = -mean_absolute_percentage_error(y, predictions)
    print("neg_mean_absolute_percentage_error", neg_mape)
    # wandb.log({'test_neg_mean_absolute_percentage_error_mean': neg_mape})

    neg_max_error = -max_error(y, predictions)
    print("neg_max_error", neg_max_error)
    # wandb.log({'test_max_error_mean': neg_max_error})

    # wandb_plot_regressor(regr, X, y)

# charts.plot_feature_importance(regr.feature_importances_, X.columns, 'GRADIENT BOOSTING')

KeyError: 'AUSSENANLAGEN'